In [1]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torchvision
from torchvision import transforms, utils

from create_dataset import ArtistsPaintingsDataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from NeuralNet import Net

from sklearn.model_selection import cross_val_predict


In [6]:
transform = transforms.Compose([transforms.CenterCrop(size=(256,256)),
                                transforms.ToTensor()])

# actual artists
# artists_idx = [8,13,15,16,19,20,22,30,31,32,46]

# debug list
artists_idx = [1,2]

train_data = ArtistsPaintingsDataset(transform = transform, mode="Train", artists_idx=artists_idx)
test_data = ArtistsPaintingsDataset(transform = transform, mode="Test", artists_idx=artists_idx)
print(len(train_data), "train images loaded.")
print(len(test_data), "test images loaded.")

train_loader = DataLoader(train_data, batch_size=256)
test_loader = DataLoader(test_data)

100%|██████████| 16/16 [00:00<00:00, 61.83it/s]

142 train images loaded.
16 test images loaded.


In [18]:
x_cv, y_cv = train_data.samples, train_data.targets
x_cv = torch.cat(x_cv, dim = 0).resize(len(y_cv),3,256,256)
print(x_cv.shape)
y_cv = torch.tensor(y_cv)
print(y_cv.shape)

torch.Size([142, 3, 256, 256])
torch.Size([142])


/home/lorenz/.local/lib/python3.8/site-packages/torch/tensor.py:365: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [23]:
from skorch import NeuralNetClassifier
from sklearn.model_selection import cross_val_score
net = NeuralNetClassifier(Net(size= (256,256), num_classes = 2),
                         train_split=None,
                         criterion = torch.nn.CrossEntropyLoss,
                         optimizer = torch.optim.SGD,
                         lr = 0.1,
                         optimizer__momentum = 0.9,
                         batch_size = 256)
scores = cross_val_score(net, x_cv, y_cv, cv=3)

  epoch    train_loss     dur
-------  ------------  ------
      1        0.6924  5.1514
      2        0.6919  3.8712
      3        0.6910  4.5421
      4        0.6900  4.0317
      5        0.6890  4.0334
      6        0.6880  3.9107
      7        0.6871  3.8359
      8        0.6863  4.1851
      9        0.6857  3.8635
     10        0.6853  3.9844
  epoch    train_loss     dur
-------  ------------  ------
      1        0.6923  4.0795
      2        0.6918  4.1024
      3        0.6908  4.2827
      4        0.6896  4.1387
      5        0.6884  4.0713
      6        0.6873  3.9425
      7        0.6863  4.0086
      8        0.6854  3.9844
      9        0.6847  3.8984
     10        0.6842  3.9448
  epoch    train_loss     dur
-------  ------------  ------
      1        0.6925  4.0000
      2        0.6921  3.9332
      3        0.6913  3.9058
      4        0.6905  3.8945
      5        0.6896  3.9763
      6        0.6888  3.9262
      7        0.6881  4.1831
      8   